# Geocoding in geopandas

Geopandas supports geocoding via a library called
[geopy](http://geopy.readthedocs.io/), which needs to be installed to use
[geopandas’ `geopandas.tools.geocode()`
function](https://geopandas.org/en/stable/docs/user_guide/geocoding.html).
`geocode()` expects a `list` or `pandas.Series` of addresses (strings) and
returns a `GeoDataFrame` with resolved addresses and point geometries.

Let’s try this out.

We will geocode addresses stored in a semicolon-separated text file called
`addresses.txt`. These addresses are located in the Helsinki Region in Southern
Finland.

In [1]:
import pathlib
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

In [2]:
import pandas
addresses = pandas.read_csv(
    DATA_DIRECTORY / "helsinki_addresses" / "addresses.txt",
    sep=";"
)

In [4]:
addresses.head()

,id,addr
0,1000,"Itämerenkatu 14, 00101 Helsinki, Finland"
1,1001,"Kampinkuja 1, 00100 Helsinki, Finland"
2,1002,"Kaivokatu 8, 00101 Helsinki, Finland"
3,1003,"Hermannin rantatie 1, 00580 Helsinki, Finland"
4,1005,"Tyynenmerenkatu 9, 00220 Helsinki, Finland"


We have an `id` for each row and an address in the `addr` column.


## Geocode addresses using *Nominatim*

In our example, we will use *Nominatim* as a *geocoding provider*. [*Nominatim*](https://nominatim.org/) is a library and service using OpenStreetMap data, and run by the OpenStreetMap Foundation. Geopandas’
[`geocode()`
function](http://geopandas.org/reference/geopandas.tools.geocode.html) supports it natively.


:::{admonition} Fair-use
:class: note

[Nominatim’s terms of use](https://operations.osmfoundation.org/policies/nominatim/)
require that users of the service make sure they don’t send more frequent
requests than one per second, and that a custom **user-agent** string is
attached to each query.

Geopandas’ implementation allows us to specify a `user_agent`; the library also
takes care of respecting the rate-limit of Nominatim.
:::

In [5]:
import geopandas

geocoded_addresses = geopandas.tools.geocode(
    addresses["addr"],
    provider="nominatim",
    user_agent="autogis2023",
    timeout=10
)
geocoded_addresses.head()

,geometry,address
0,POINT (24.91556 60.16320),"Ruoholahti, 14, Itämerenkatu, Salmisaari, Ruoh..."
1,POINT (24.93166 60.16905),"Kamppi, 1, Kampinkuja, Kamppi, Eteläinen suurp..."
2,POINT (24.94179 60.16989),"Kauppakeskus Citycenter, 8, Kaivokatu, Keskust..."
3,POINT (24.97846 60.19032),"Hermannin rantatie, Verkkosaari, Kalasatama, S..."
4,POINT (24.92169 60.15667),"9, Tyynenmerenkatu, Jätkäsaari, Länsisatama, E..."


Et voilà! As a result we received a `GeoDataFrame` that contains a parsed
version of our original addresses and a `geometry` column of
`shapely.geometry.Point`s that we can use, for instance, to export the data to
a geospatial data format.

However, the `id` column was discarded in the process. To combine the input
data set with our result set, we can use pandas’ [*join*
operations](https://pandas.pydata.org/docs/user_guide/merging.html).


## Join data frames

:::{admonition} Joining data sets using pandas
:class: note

For a comprehensive overview of different ways of combining DataFrames and
Series based on set theory, have a look at pandas documentation about [merge,
join and
concatenate](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html).
:::


Joining data from two or more data frames or tables is a common task in many
(spatial) data analysis workflows. As you might remember from our earlier
lessons, combining data from different tables based on common **key** attribute
can be done easily in pandas/geopandas using the [`merge()`
function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html).
We used this approach in [exercise 6 of the Geo-Python
course](https://geo-python-site.readthedocs.io/en/latest/lessons/L6/exercise-6.html#joining-data-from-one-dataframe-to-another).

However, sometimes it is useful to join two data frames together based on their
**index**. The data frames have to have the **same number of records** and
**share the same index** (simply put, they should have the same order of rows).

We can use this approach, here, to join information from the original data
frame `addresses` to the geocoded addresses `geocoded_addresses`, row by row.
The `join()` function, by default, joins two data frames based on their index.
This works correctly for our example, as the order of the two data frames is
identical.

In [6]:
geocoded_addresses_with_id = geocoded_addresses.join(addresses)
geocoded_addresses_with_id

,geometry,address,id,addr
0,POINT (24.91556 60.16320),"Ruoholahti, 14, Itämerenkatu, Salmisaari, Ruoh...",1000,"Itämerenkatu 14, 00101 Helsinki, Finland"
1,POINT (24.93166 60.16905),"Kamppi, 1, Kampinkuja, Kamppi, Eteläinen suurp...",1001,"Kampinkuja 1, 00100 Helsinki, Finland"
2,POINT (24.94179 60.16989),"Kauppakeskus Citycenter, 8, Kaivokatu, Keskust...",1002,"Kaivokatu 8, 00101 Helsinki, Finland"
3,POINT (24.97846 60.19032),"Hermannin rantatie, Verkkosaari, Kalasatama, S...",1003,"Hermannin rantatie 1, 00580 Helsinki, Finland"
4,POINT (24.92169 60.15667),"9, Tyynenmerenkatu, Jätkäsaari, Länsisatama, E...",1005,"Tyynenmerenkatu 9, 00220 Helsinki, Finland"
5,POINT (25.08174 60.23522),"18, Kontulantie, Kontula, Mellunkylä, Itäinen ...",1006,"Kontulantie 18, 00940 Helsinki, Finland"
6,POINT (25.10985 60.22126),"Itäväylä, Vartioharju, Vartiokylä, Itäinen suu...",1007,"Itäväylä 3, 00950 Helsinki, Finland"
7,POINT (25.02180 60.27824),"Tapulikaupungintie, Ala-Tikkurila, Tapulikaupu...",1008,"Tapulikaupungintie 3, 00750 Helsinki, Finland"
8,POINT (25.02878 60.26322),"Sompionpolku, Fallkullan kiila, Tapanila, Tapa...",1009,"Sompionpolku 2, 00730 Helsinki, Finland"
9,POINT (24.87197 60.22244),"5, Atomitie, Strömberg, Pitäjänmäen teollisuus...",1010,"Atomitie 5, 00370 Helsinki, Finland"


The output of `join()` is a new `geopandas.GeoDataFrame`:

In [7]:
type(geocoded_addresses_with_id)

geopandas.geodataframe.GeoDataFrame

The new data frame has all original columns plus new columns for the `geometry`
and for a parsed `address` that can be used to spot-check the results.

:::{note}
If you would do the join the other way around, i.e. `addresses.join(geocoded_addresses)`, the output would be a `pandas.DataFrame`, not a `geopandas.GeoDataFrame`.
:::


---


It’s now easy to save the new data set as a geospatial file, for instance, in
*GeoPackage* format:

In [8]:
geocoded_addresses.to_file(DATA_DIRECTORY / "addresses.gpkg")